OJO: PUEDE HABER DIFERENCIAS ENTRE EL CÓDIGO DEL .PY Y DEL NOTEBOOK. EL BUENEO ES EL DEL .PY

In [1]:

import numpy as np
import pandas as pd
import nbformat
import re
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import openai
from openai import RateLimitError, OpenAIError
from openpyxl import Workbook
from openpyxl.styles import Font
import os
from dotenv import load_dotenv
from pprint import pprint
from openai import OpenAI
from fpdf import FPDF
from tqdm import tqdm
import sys
from joblib import Parallel, delayed
import logging

In [2]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key is None:
    raise ValueError("API key is not set")

# Inicializar la API de OpenAI
openai.api_key = openai_api_key

In [3]:
def configurar_logger(log_file):
    # Comprobar si el logger ya está configurado
    if not logging.getLogger().hasHandlers():
        # Configurar el logger una vez
        logging.basicConfig(filename=log_file, level=logging.ERROR, 
                            format='%(asctime)s - %(message)s')

In [4]:
def listar_notebooks(directory, log_file):
    """
    Devuelve una lista con los nombres de todos los archivos con extensión '.ipynb' en un directorio dado.
    
    Parameters:
        directory (str): La ruta al directorio donde buscar los archivos.
        log_file (str): La ruta al archivo de log donde se registrarán los mensajes.

    Returns:
        tuple: Dos listas, la primera con los nombres de los alumnos (nombres de archivos sin extensión),
               y la segunda con los nombres de los archivos completos que cumplen los criterios.
    """
    # Configurar el logging
    configurar_logger(log_file)

    archivos = []
    try:
        for filename in os.listdir(directory):
            if filename.endswith('.ipynb'):
                archivos.append(filename)
        
        alumnos = [archivo.replace('.ipynb', '') for archivo in archivos]
        
        if not archivos:
            logging.warning(f"No se encontraron archivos .ipynb en el directorio {directory}'.")
            print(f"No se encontraron archivos .ipynb en el directorio {directory}'.")
        
        return alumnos, archivos

    except Exception as e:
        error_msg = f"Error al listar archivos en el directorio {directory}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)


In [5]:
def cargar_criterios(criterios_file, log_file):
    """
    Carga los criterios desde un archivo de texto y los almacena en un diccionario.

    Parameters:
        criterios_file (str): Ruta del archivo de texto que contiene los criterios.
        log_file (str): Ruta del archivo donde se guardarán los logs de errores.

    Returns:
        dict: Un diccionario donde las claves son los nombres de los criterios y los valores
              son diccionarios con 'descripcion' y 'ejemplo'.

    Raises:
        RuntimeError: Si ocurre un error crítico durante la carga de criterios.
    """
    criterios = {}

    # Configurar el logging para registrar solo errores
    configurar_logger(log_file)
    
    try:
        # Intentar abrir el archivo de criterios y leer su contenido
        with open(criterios_file, 'r', encoding='utf-8') as file:
            contenido = file.read()
    except FileNotFoundError:
        # Registrar un error si el archivo no se encuentra
        error_msg = f"Error en cargar_criterios: El archivo {criterios_file} no se encontró."
        logging.error(error_msg)
        raise RuntimeError(error_msg)
    except IOError as e:
        # Registrar cualquier otro error de entrada/salida
        error_msg = f"Error en cargar_criterios: Error al leer el archivo {criterios_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    try:
        # Dividir el contenido en secciones usando '@@' como delimitador
        secciones = contenido.split('@@')

        # Validar que el contenido esté correctamente estructurado
        if len(secciones) < 3 or len(secciones) % 2 == 0:
            raise ValueError(f"Error en cargar_criterios: Formato incorrecto en el archivo {criterios_file}. Verifique que cada criterio tenga nombre y detalles asociados.")

        # Iterar sobre las secciones para extraer los criterios y sus detalles
        for i in range(1, len(secciones), 2):
            nombre_criterio = secciones[i].strip()  # Extraer y limpiar el nombre del criterio
            detalles = secciones[i + 1].strip()  # Extraer y limpiar los detalles del criterio
            partes = detalles.split("Ejemplo:")  # Dividir detalles en descripción y ejemplo
            descripcion = partes[0].replace("Descripción:", "").strip()  # Limpiar la descripción
            ejemplo = partes[1].strip() if len(partes) > 1 else ""  # Limpiar el ejemplo si existe

            # Almacenar el criterio en el diccionario
            criterios[nombre_criterio] = {"descripcion": descripcion, "ejemplo": ejemplo}

        # Si no se encontraron criterios válidos, lanzar una excepción
        if not criterios:
            raise ValueError(f"Error en cargar_criterios: No se encontraron criterios válidos en el archivo {criterios_file}.")

    except IndexError:
        # Registrar un error si el formato del archivo es incorrecto
        error_msg = "Error en cargar_criterios: Formato incorrecto en el archivo de criterios. Verifique la estructura del archivo."
        logging.error(error_msg)
        raise RuntimeError(error_msg)
    except ValueError as e:
        # Registrar cualquier error relacionado con los criterios
        logging.error(f"Error en cargar_criterios: {e}")
        raise RuntimeError(f"Error en cargar_criterios: {e}")
    except Exception as e:
        # Registrar cualquier otro error inesperado
        error_msg = f"Error inesperado en cargar_criterios al procesar el archivo {criterios_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    # Log de resultado final
    logging.info("Criterios cargados correctamente. No se encontraron errores.")
    
    # Devolver el diccionario de criterios cargado correctamente
    return criterios


In [6]:
def verifica_estructura_examen(examen_file, log_file):
    """
    Verifica que un notebook Jupyter sigue la estructura esperada para un examen.

    Parameters:
        examen_file (str): Ruta del archivo del notebook de examen.
        log_file (str): Ruta del archivo donde se guardarán los logs de errores.

    Returns:
        None

    Raises:
        RuntimeError: Si se detecta un error crítico en la estructura.
    """
    errores = []
    contexto_detectado = False
    ejercicio_num = 0
    se_espera_solucion = False
    codigo_encontrado = False

    # Configurar el logging para registrar solo errores
    configurar_logger(log_file)
    
    try:
        # Leer el notebook
        with open(examen_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
    except FileNotFoundError:
        error_msg = f"Error en verifica_estructura_examen: El archivo {examen_file} no se encontró."
        logging.error(error_msg)
        raise RuntimeError(error_msg)
    except Exception as e:
        error_msg = f"Error en verifica_estructura_examen: Error al leer el archivo {examen_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    try:
        # Procesar las celdas del notebook
        for cell in notebook.cells:
            if cell.cell_type == 'markdown':
                # Si se esperaba una solución y no se encontró código, generar un error
                if se_espera_solucion and not codigo_encontrado:
                    error_msg = f"Error en verifica_estructura_examen: La Solución del Ejercicio {ejercicio_num} no contiene código."
                    errores.append(error_msg)
                    logging.error(error_msg)
                se_espera_solucion = False
                codigo_encontrado = False

                cell_content = cell['source'].strip()

                # Verificar Contexto
                if cell_content.startswith("## Contexto"):
                    if contexto_detectado:
                        error_msg = "Error en verifica_estructura_examen: Más de un '## Contexto' detectado."
                        errores.append(error_msg)
                        logging.error(error_msg)
                    contexto_detectado = True

                # Verificar Ejercicios y Criterios
                if cell_content.startswith("## Ejercicio"):
                    ejercicio_num += 1
                    se_espera_solucion = True

                    if "Criterios:" not in cell_content:
                        error_msg = f"Error en verifica_estructura_examen: El Ejercicio {ejercicio_num} no contiene la sección 'Criterios'."
                        errores.append(error_msg)
                        logging.error(error_msg)
                    else:
                        criterios = cell_content.split("Criterios:")[-1].strip()
                        criterios_list = criterios.split(",")

                        # Verificar si cada criterio está delimitado correctamente y no está vacío
                        for criterio in criterios_list:
                            criterio = criterio.strip()
                            if not (criterio.startswith("@@") and criterio.endswith("@@")) or len(criterio) <= 4:
                                error_msg = f"Error en verifica_estructura_examen: Criterios mal formateados o vacíos en el Ejercicio {ejercicio_num}"
                                errores.append(error_msg)
                                logging.error(error_msg)
                                break

            elif cell.cell_type == 'code':
                if se_espera_solucion:
                    cell_content = cell['source'].strip()

                    # Eliminar saltos de línea y espacios adicionales para una comparación más robusta
                    normalized_content = " ".join(cell_content.split())

                    # Verificar si la celda comienza con "## Solución ejercicio" o "## Solucion ejercicio"
                    if normalized_content.startswith("## Solución ejercicio") or normalized_content.startswith("## Solucion ejercicio"):
                        try:
                            sol_num = int(normalized_content.split("## Solución ejercicio" if "## Solución ejercicio" in normalized_content else "## Solucion ejercicio")[1].strip().split()[0])
                            if sol_num != ejercicio_num:
                                error_msg = f"Error en verifica_estructura_examen: La Solución ejercicio {sol_num} no corresponde al Ejercicio {ejercicio_num}."
                                errores.append(error_msg)
                                logging.error(error_msg)
                            codigo_encontrado = False  # Reiniciar el indicador de código encontrado
                        except (ValueError, IndexError):
                            error_msg = f"Error en verifica_estructura_examen: Formato de número incorrecto en la Solución del Ejercicio {ejercicio_num}."
                            errores.append(error_msg)
                            logging.error(error_msg)
                    else:
                        # Si la celda no comienza con "## Solución ejercicio" ni "## Solucion ejercicio", pero se espera una solución
                        error_msg = f"Error en verifica_estructura_examen: El Ejercicio {ejercicio_num} tiene una celda de código que no comienza con '## Solución ejercicio {ejercicio_num}' o '## Solucion ejercicio {ejercicio_num}'."
                        errores.append(error_msg)
                        logging.error(error_msg)
                        se_espera_solucion = False

                    # Verificar si la celda contiene código más allá de comentarios o está vacía
                    lines = cell_content.split("\n")
                    for line in lines:
                        stripped_line = line.strip()
                        if stripped_line and not stripped_line.startswith("#"):
                            codigo_encontrado = True
                            break

        # Verificar si la última solución esperada fue proporcionada y si tenía código
        if se_espera_solucion and not codigo_encontrado:
            error_msg = f"Error en verifica_estructura_examen: La Solución del Ejercicio {ejercicio_num} no contiene código."
            errores.append(error_msg)
            logging.error(error_msg)

        # Verificaciones finales
        if not contexto_detectado:
            error_msg = "Error en verifica_estructura_examen: No se detectó un '## Contexto' en el notebook."
            errores.append(error_msg)
            logging.error(error_msg)

    except Exception as e:
        error_msg = f"Error inesperado en verifica_estructura_examen al procesar el archivo {examen_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    # Log de resultado final
    if errores:
        logging.error("EXAMEN Estructura Incorrecta: Se encontraron los siguientes errores:")
        for error in errores:
            logging.error(error)
        raise RuntimeError("EXAMEN Se encontraron errores críticos en la estructura del notebook.")
    else:
        logging.error("EXAMEN Estructura Correcta: El notebook sigue la estructura esperada.")

### COMPROBACIÓN DE LAS DOS FUNCIONES cargar_criterios y verifica_estructura_examen

In [7]:

directorio_raiz = "/workspace"
directorio_entregas = os.path.join(directorio_raiz, "entregas")
dir_log = os.path.join(directorio_raiz, "logs")
prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
directorio_examen = os.path.join(directorio_raiz, "examenes")
directorio_reports = os.path.join(directorio_raiz, "reports")

log_file = os.path.join(dir_log, 'evaluacion.log')
nombre_fich_examen = 'examen_pruebas.ipynb'
criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
fichero_res = 'resultados_evaluacion.xlsx'
examen_file = os.path.join(directorio_examen, nombre_fich_examen)


# Borrar el archivo de log si ya existe para empezar con un archivo limpio
if os.path.exists(log_file):
    os.remove(log_file)

# Configurar el logging para registrar errores tanto en consola como en el archivo de log
logging.basicConfig(filename=log_file, level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # Llamar a la función cargar_criterios
    criterios = cargar_criterios(criterios_file, log_file)
    print("Criterios cargados correctamente. Continuando con la verificación del examen...")

except RuntimeError as e:
    print(f"Error crítico al cargar los criterios: {e}")
    sys.exit(1)  # Termina el script con un código de error

try:
    # Llamar a la función verifica_estructura_examen
    verifica_estructura_examen(examen_file, log_file)
    print("La estructura del examen es correcta. Script completado con éxito.")

except RuntimeError as e:
    print(f"Error crítico al verificar la estructura del examen: {e}")
    sys.exit(1)  # Termina el script con un código de error

Criterios cargados correctamente. Continuando con la verificación del examen...
La estructura del examen es correcta. Script completado con éxito.


In [8]:
def extrae_informacion_examen(examen_file, log_file):
    """
    Extrae la información del examen desde un notebook Jupyter.

    Parameters:
        examen_file (str): Ruta del archivo del notebook de examen.
        log_file (str): Ruta del archivo donde se guardarán los logs de errores.
        
    Returns:
        dict: Un diccionario con el 'contexto' del examen y una lista de 'ejercicios', 
              donde cada ejercicio tiene 'enunciado', 'criterios', y 'solucion'.

    Raises:
        RuntimeError: Si ocurre un error crítico durante la extracción de información.
    """
    examen_info = {
        "contexto": "",
        "ejercicios": []
    }

    contexto_detectado = False
    ejercicio_num = 0
    se_espera_solucion = False
    solucion_detectada = False
    ejercicio_info = {}

    configurar_logger(log_file)

    try:
        # Leer el notebook
        with open(examen_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
        logging.error(f"Notebook {examen_file} leído correctamente.")
    except FileNotFoundError:
        error_msg = f"Error en extrae_informacion_examen: El archivo {examen_file} no se encontró."
        logging.error(error_msg)
        raise RuntimeError(error_msg)
    except Exception as e:
        error_msg = f"Error en extrae_informacion_examen al leer el archivo {examen_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    try:
        # Procesar las celdas del notebook
        for cell in notebook.cells:
            if cell.cell_type == 'markdown':
                cell_content = cell['source'].strip()

                # Extraer Contexto
                if cell_content.startswith("## Contexto"):
                    examen_info["contexto"] = cell_content.split("## Contexto:")[-1].strip()
                    contexto_detectado = True
                    logging.error("Contexto del examen detectado y extraído correctamente.")

                # Extraer Enunciado y Criterios del Ejercicio
                if cell_content.startswith("## Ejercicio"):
                    if ejercicio_num > 0:
                        examen_info["ejercicios"].append(ejercicio_info)
                        logging.error(f"Ejercicio {ejercicio_num} procesado y añadido a la lista.")

                    ejercicio_num += 1
                    se_espera_solucion = True
                    solucion_detectada = False
                    ejercicio_info = {
                        "enunciado": cell_content.split("## Ejercicio")[1].split("Criterios:")[0].strip(),
                        "criterios": [],
                        "solucion": []
                    }
                    criterios_text = cell_content.split("Criterios:")[-1].strip()
                    criterios_list = [criterio.strip() for criterio in criterios_text.split(",")]
                    ejercicio_info["criterios"] = criterios_list
                    logging.error(f"Ejercicio {ejercicio_num} detectado con enunciado y criterios extraídos.")

            elif cell.cell_type == 'code' and se_espera_solucion:
                # Extraer solución de las celdas de código
                ejercicio_info["solucion"].append(cell['source'].strip())
                solucion_detectada = True
                logging.error(f"Solución del Ejercicio {ejercicio_num} extraída y añadida.")

        # Agregar la información del último ejercicio si no ha sido añadido
        if ejercicio_num > 0 and ejercicio_info:
            examen_info["ejercicios"].append(ejercicio_info)
            logging.error(f"Ejercicio {ejercicio_num} procesado y añadido a la lista.")

        if not contexto_detectado:
            error_msg = "Error en extrae_informacion_examen: No se detectó un '## Contexto' en el notebook."
            logging.error(error_msg)
            raise RuntimeError(error_msg)

        logging.error("La información del examen ha sido extraída correctamente.")

    except Exception as e:
        error_msg = f"Error inesperado en extrae_informacion_examen al procesar el archivo {examen_file}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    return examen_info



In [10]:
def procesa_respuestas_alumno(alumno_file, numero_ejercicios, log_file):
    """
    Procesa un notebook de respuestas de un alumno para extraer las soluciones a los ejercicios.

    Parameters:
        alumno_file (str): Ruta del archivo del notebook de respuesta del alumno.
        numero_ejercicios (int): Número esperado de ejercicios en el examen.
        log_file (str): Ruta del archivo de log donde se guardarán los mensajes de error.

    Returns:
        dict: Un diccionario con el 'nombre_alumno' y una lista de 'ejercicios', donde cada ejercicio
              tiene su 'enunciado' y 'solucion' (o un valor indicando que no fue respondido).
    """
    # Extraer el nombre del alumno desde el nombre del archivo
    nombre_alumno = os.path.splitext(os.path.basename(alumno_file))[0]

    # Configurar el logging
    configurar_logger(log_file)

    # Iniciar el log indicando el inicio del procesamiento
    logging.error(f"Procesando respuestas del alumno {nombre_alumno}")

    respuestas_alumno = {
        "nombre_alumno": nombre_alumno,
        "ejercicios": []
    }

    ejercicio_num = 0
    se_espera_solucion = False
    ejercicio_info = {}

    try:
        # Leer el notebook
        with open(alumno_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
        logging.error(f"Notebook {alumno_file} leído correctamente para el alumno {nombre_alumno}.")

    except FileNotFoundError:
        error_msg = f"Error en procesa_respuestas_alumno: Archivo no encontrado para el alumno {nombre_alumno}: {alumno_file}"
        logging.error(error_msg)
        return {"error": "Archivo no encontrado."}
    except Exception as e:
        error_msg = f"Error en procesa_respuestas_alumno al leer el notebook para el alumno {nombre_alumno}: {e}"
        logging.error(error_msg)
        return {"error": f"Error al leer el notebook: {str(e)}"}

    try:
        # Procesar las celdas del notebook
        for cell in notebook.cells:
            if cell.cell_type == 'markdown':
                cell_content = cell['source'].strip()

                # Identificar un nuevo ejercicio basado en el enunciado
                if cell_content.startswith("## Ejercicio"):
                    # Si estábamos esperando una solución y no se añadió código, marcamos como no respondido
                    if se_espera_solucion:
                        if not ejercicio_info.get("solucion") or all(
                            not line.strip() or line.strip().startswith("#") 
                            for solution in ejercicio_info["solucion"] 
                            for line in solution.splitlines()):
                            ejercicio_info["solucion"] = "No respondido"
                        respuestas_alumno["ejercicios"].append(ejercicio_info)
                        logging.error(f"Ejercicio {ejercicio_num} procesado para el alumno {nombre_alumno}.")

                    ejercicio_num += 1
                    se_espera_solucion = True
                    ejercicio_info = {
                        "enunciado": cell_content.split("## Ejercicio")[1].strip(),
                        "solucion": []
                    }
                    logging.error(f"Ejercicio {ejercicio_num} detectado para el alumno {nombre_alumno}.")

            elif cell.cell_type == 'code' and se_espera_solucion:
                # Procesar celdas de código para una solución
                codigo = cell['source'].strip()
                if codigo:
                    ejercicio_info["solucion"].append(codigo)
                    logging.error(f"Solución del Ejercicio {ejercicio_num} extraída para el alumno {nombre_alumno}.")

        # Agregar la información del último ejercicio si no ha sido añadido
        if se_espera_solucion:
            if not ejercicio_info.get("solucion") or all(
                not line.strip() or line.strip().startswith("#") 
                for solution in ejercicio_info["solucion"] 
                for line in solution.splitlines()):
                ejercicio_info["solucion"] = "No respondido"
            respuestas_alumno["ejercicios"].append(ejercicio_info)
            logging.error(f"Ejercicio {ejercicio_num} procesado para el alumno {nombre_alumno}.")

        # Verificar que el número de ejercicios y soluciones coincida con el número esperado
        num_ejercicios = len(respuestas_alumno["ejercicios"])
        num_soluciones = sum(1 for ejercicio in respuestas_alumno["ejercicios"] if ejercicio["solucion"] != "No respondido")

        if num_ejercicios != numero_ejercicios:
            error_msg = f"Error en procesa_respuestas_alumno: El número de ejercicios en el notebook ({num_ejercicios}) no coincide con el número esperado ({numero_ejercicios}) para el alumno {nombre_alumno}."
            logging.error(error_msg)
            return {"error": error_msg}

        if num_soluciones != numero_ejercicios:
            error_msg = f"Error en procesa_respuestas_alumno: El número de soluciones en el notebook ({num_soluciones}) no coincide con el número esperado de ejercicios ({numero_ejercicios}) para el alumno {nombre_alumno}."
            logging.error(error_msg)
            return {"error": error_msg}

        logging.error(f"Respuestas del alumno {nombre_alumno} procesadas correctamente.")

    except Exception as e:
        error_msg = f"Error inesperado en procesa_respuestas_alumno al procesar el archivo {alumno_file} para el alumno {nombre_alumno}: {e}"
        logging.error(error_msg)
        return {"error": f"Error inesperado: {str(e)}"}

    return respuestas_alumno



In [11]:
def comprueba_ejecucion(respuestas_alumno, log_file):
    """
    Comprueba si las soluciones de los ejercicios se ejecutan sin errores y añade el estado
    y el mensaje de error (si corresponde) al diccionario original de respuestas del alumno.

    Parameters:
        respuestas_alumno (dict): Diccionario que contiene las respuestas del alumno y sus soluciones.
        log_file (str): Ruta del archivo de log donde se guardarán los mensajes de error.
        
    Returns:
        dict: El mismo diccionario 'respuestas_alumno' con los campos 'estado' y 'mensaje_de_error'
              añadidos a cada ejercicio.

    Raises:
        RuntimeError: Si ocurre un error crítico durante la ejecución de la función.
    """
    # Configurar el logging
    configurar_logger(log_file)

    nombre_alumno = respuestas_alumno["nombre_alumno"]

    try:
        for i, ejercicio in enumerate(respuestas_alumno["ejercicios"], 1):
            solucion = ejercicio["solucion"]

            if solucion == "No respondido":
                ejercicio["estado"] = "No respondido"
                ejercicio["mensaje_de_error"] = None
            else:
                # Redirigir la salida estándar a un objeto StringIO para capturarla
                original_stdout = sys.stdout
                sys.stdout = io.StringIO()

                try:
                    # Crear un diccionario para almacenar el entorno de ejecución
                    entorno_ejecucion = {}
                    # Concatenar todos los bloques de código en una sola cadena
                    codigo_completo = "\n".join(solucion)
                    # Ejecutar el código completo en el entorno de ejecución
                    exec(codigo_completo, entorno_ejecucion)
                    ejercicio["estado"] = "Correcto"
                    ejercicio["mensaje_de_error"] = None
                except Exception as e:
                    # Si hay un error en la ejecución, marcar el estado como "Error" y guardar el mensaje de error
                    ejercicio["estado"] = "Error"
                    ejercicio["mensaje_de_error"] = str(e)
                finally:
                    # Restaurar la salida estándar original
                    sys.stdout = original_stdout

    except Exception as e:
        error_msg = f"Error inesperado en comprueba_ejecucion para el alumno {nombre_alumno}: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    return respuestas_alumno



## COMPROBAR LAS FUNCIONES cargar_criterios, verifica_estructura_examen, extrae_informacion_examen, procesa_respuestas_alumno, comprueba_ejecucion

In [12]:
import os
import logging
import sys
import io

# Configurar las rutas de los archivos y directorios
directorio_raiz = "/workspace"
directorio_entregas = os.path.join(directorio_raiz, "entregas")
dir_log = os.path.join(directorio_raiz, "logs")
prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
directorio_examen = os.path.join(directorio_raiz, "examenes")
directorio_reports = os.path.join(directorio_raiz, "reports")

log_file = os.path.join(dir_log, 'evaluacion.log')
nombre_fich_examen = 'examen_pruebas.ipynb'
criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
fichero_res = 'resultados_evaluacion.xlsx'
examen_file = os.path.join(directorio_examen, nombre_fich_examen)

# Crear el directorio de logs si no existe
if not os.path.exists(dir_log):
    os.makedirs(dir_log)

# Borrar el archivo de log si ya existe para empezar con un archivo limpio
if os.path.exists(log_file):
    os.remove(log_file)


# Configurar el logging para registrar errores tanto en consola como en el archivo de log
logging.basicConfig(filename=log_file, level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # 1. Cargar criterios
    criterios = cargar_criterios(criterios_file, log_file)
    print("Criterios cargados correctamente. Continuando con la verificación del examen...")

    # 2. Verificar la estructura del examen
    verifica_estructura_examen(examen_file, log_file)
    print("La estructura del examen es correcta. Continuando con la extracción de información...")

    # 3. Extraer la información del examen
    examen_info = extrae_informacion_examen(examen_file, log_file)
    print("Información del examen extraída correctamente. Continuando con el procesamiento de respuestas del alumno...")

    # 4. Procesar respuestas de un alumno
    nombre_fich_alumno = 'ubeda_fernando.ipynb'
    alumno_file = os.path.join(directorio_entregas, nombre_fich_alumno)
    respuestas_alumno = procesa_respuestas_alumno(alumno_file, len(examen_info['ejercicios']), log_file)
    print("Respuestas del alumno procesadas correctamente. Continuando con la comprobación de ejecución...")

    # 5. Comprobar la ejecución de las respuestas del alumno
    respuestas_evaluadas = comprueba_ejecucion(respuestas_alumno, log_file)
    print("Ejecución de las respuestas del alumno verificada correctamente.")

    # 6. Imprimir un resumen del resultado
    print("Resumen de la evaluación:")
    print(f"Nombre del alumno: {respuestas_evaluadas['nombre_alumno']}")
    for i, ejercicio in enumerate(respuestas_evaluadas['ejercicios'], 1):
        estado = ejercicio.get('estado', 'No definido')
        mensaje_error = ejercicio.get('mensaje_de_error', 'Ninguno')
        print(f"Ejercicio {i}: Estado - {estado}, Mensaje de error - {mensaje_error}")

except RuntimeError as e:
    print(f"Error crítico: {e}")
    sys.exit(1)  # Termina el script con un código de error


Criterios cargados correctamente. Continuando con la verificación del examen...
La estructura del examen es correcta. Continuando con la extracción de información...
Información del examen extraída correctamente. Continuando con el procesamiento de respuestas del alumno...
Respuestas del alumno procesadas correctamente. Continuando con la comprobación de ejecución...
Ejecución de las respuestas del alumno verificada correctamente.
Resumen de la evaluación:
Nombre del alumno: ubeda_fernando
Ejercicio 1: Estado - Error, Mensaje de error - list assignment index out of range
Ejercicio 2: Estado - Error, Mensaje de error - cannot access local variable 'maximo' where it is not associated with a value
Ejercicio 3: Estado - Error, Mensaje de error - name 'valor_total' is not defined


In [41]:
respuestas_alumno

{'nombre_alumno': 'ubeda_fernando',
 'ejercicios': [{'enunciado': '1: Datos de Ingresos y Gastos.\n\nDefine una lista llamada ingresos que contenga los ingresos mensuales de una empresa durante 6 meses. Por ejemplo: [5000, 5500, 6000, 6500, 7000, 7500].\n\nDefine una lista llamada gastos que contenga los gastos mensuales correspondientes a esos mismos 6 meses. Por ejemplo: [3000, 3200, 3500, 3700, 3800, 3900].\n\nCalcula el saldo mensual (ingresos menos gastos) y almacena los saldos en una lista llamada saldos.\n\nImprime la lista de saldos y el saldo total acumulado de los 6 meses.',
   'solucion': ['## Solucion ejercicio 1\n\ningresos = [5000, 5500, 6000, 6500, 7000, 7500]\ngastos = [3000, 3200, 3500, 3700, 3800, 3900]\nsaldos = []\n\n# Escribe aqui tu codigo\nsaldos[0] = ingresos[0] - gastos[0]\nsaldos[1] = ingresos[1] - gastos[1]\nsaldos[2] = ingresos[2] - gastos[2]\nsaldos[3] = ingresos[3] - gastos[3]\nsaldos[4] = ingresos[4] - gastos[4]\n\nprint("Saldos mensuales:", saldos)\nprin

_________________________________________________

In [13]:
def evaluar_con_chatgpt(contexto, codigo, enunciado, criterios_texto, prompt_template, log_file):
    """
    Evalúa el código de un ejercicio utilizando el modelo GPT-4 de OpenAI.

    Parameters:
        contexto (str): El contexto del examen.
        codigo (str): El código del ejercicio a evaluar.
        enunciado (str): El enunciado del ejercicio.
        criterios_texto (str): Texto que contiene los criterios específicos para este ejercicio.
        prompt_template (str): El template del prompt con marcadores para la descripción y el código.
        log_file (str): La ruta del archivo de log donde se guardarán los mensajes de error.

    Returns:
        dict: Un diccionario con la puntuación y comentario del ejercicio.
    """
    # Configurar el logging
    configurar_logger(log_file)

    # Insertar los valores en el prompt, incluyendo los criterios específicos para este ejercicio
    prompt = prompt_template.format(contexto=contexto, enunciado=enunciado, codigo=codigo, criterios=criterios_texto)

    try:
        cliente = OpenAI()
        response = cliente.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a programming teaching assistant evaluating student code."},
                {"role": "user", "content": prompt}
            ]
        )
    except RateLimitError as e:
        error_msg = f"Rate limit error: {e}"
        logging.error(error_msg)
        return {"error": error_msg}
    except OpenAIError as e:
        error_msg = f"OpenAI API error: {e}"
        logging.error(error_msg)
        return {"error": error_msg}
    except Exception as e:
        error_msg = f"General error: {e}"
        logging.error(error_msg)
        return {"error": error_msg}

    # Extraer la respuesta de ChatGPT
    try:
        evaluacion = response.choices[0].message.content
    except (KeyError, IndexError) as e:
        error_msg = f"Error al procesar la respuesta de la API: {e}"
        logging.error(error_msg)
        return {"error": error_msg}

    return evaluacion



In [14]:
def evaluar_ejercicios(info_examen, resultados_alumno, prompt_file, criterios_file, log_file):
    """
    Evalúa los ejercicios utilizando GPT-4 y devuelve las notas y comentarios para cada ejercicio.
    
    Parameters:
        info_examen (dict): Diccionario con el contexto del examen y los enunciados de los ejercicios, obtenido de `extrae_informacion_examen`.
        resultados_alumno (dict): Diccionario con las soluciones y estado de ejecución de los ejercicios, obtenido de `comprueba_ejecucion`.
        prompt_file (str): Ruta del archivo de texto que contiene el prompt.
        criterios_file (str): Ruta del archivo de texto que contiene los criterios.
        log_file (str): Ruta del archivo de log donde se guardarán los mensajes de error.

    Returns:
        dict: Un diccionario con las evaluaciones de cada ejercicio.

    Raises:
        RuntimeError: Si ocurre un error crítico durante la evaluación.
    """
    # Configurar el logging
    configurar_logger(log_file)

    evaluaciones = {}

    try:
        # Leer el prompt desde el archivo
        with open(prompt_file, 'r', encoding='utf-8') as file:
            prompt_template = file.read()

        # Cargar los criterios desde el archivo
        criterios_info = cargar_criterios(criterios_file, log_file)

        # Iterar sobre los ejercicios y sus resultados
        for i, (ejercicio_info, resultado_alumno) in enumerate(zip(info_examen['ejercicios'], resultados_alumno['ejercicios']), start=1):
            enunciado = ejercicio_info['enunciado']
            codigo = resultado_alumno['solucion']
            criterios_nombres = ejercicio_info.get('criterios', [])

            # Generar el texto de los criterios para el prompt
            criterios_texto = "\n\n".join([
                f"**{criterio.strip('@')}**\nDescripción: {criterios_info[criterio.strip('@')]['descripcion']}\nEjemplo: {criterios_info[criterio.strip('@')]['ejemplo']}"
                for criterio in criterios_nombres if criterio.strip('@') in criterios_info
            ])

            # Redirigir la salida estándar a un objeto StringIO para capturarla
            original_stdout = sys.stdout
            sys.stdout = io.StringIO()

            try:
                # Si el ejercicio no fue respondido, se añade una evaluación específica
                if resultado_alumno['estado'] == "No respondido":
                    evaluaciones[f'Ejercicio {i}'] = (
                        "**Puntuaciones**: [0]\n"
                        "**Comentarios**: [\"El ejercicio no fue respondido.\"]\n"
                        "**Comentario General**: [\"El alumno no proporcionó ninguna solución para este ejercicio.\"]"
                    )
                elif resultado_alumno['estado'] == "Error":
                    evaluaciones[f'Ejercicio {i}'] = (
                        "**Puntuaciones**: [0]\n"
                        f"**Comentarios**: [\"Error en la ejecución: {resultado_alumno['mensaje_de_error']}\"]\n"
                        "**Comentario General**: [\"El código presentado contiene errores que impiden su correcta ejecución.\"]"
                    )
                else:
                    # Llamar a la función que evalúa con ChatGPT pasando el prompt template y los criterios específicos
                    resultado = evaluar_con_chatgpt(
                        info_examen['contexto'], codigo, enunciado, criterios_texto, prompt_template, log_file
                    )
                    evaluaciones[f'Ejercicio {i}'] = resultado

            except Exception as e:
                error_msg = f"Error al evaluar el ejercicio {i} para el alumno: {e}"
                logging.error(error_msg)
                evaluaciones[f'Ejercicio {i}'] = (
                    "**Puntuaciones**: [0]\n"
                    f"**Comentarios**: [\"Error durante la evaluación: {str(e)}\"]\n"
                    "**Comentario General**: [\"Ocurrió un error inesperado durante la evaluación del código.\"]"
                )

            finally:
                # Restaurar la salida estándar original
                sys.stdout = original_stdout

    except Exception as e:
        error_msg = f"Error inesperado en evaluar_ejercicios: {e}"
        logging.error(error_msg)
        raise RuntimeError(error_msg)

    return evaluaciones



In [15]:
import re
import logging
import os

def extraer_resultados(resultados, nombre_alumno, log_file):
    """
    Extrae las puntuaciones, comentarios y comentarios generales de una estructura de resultados para todos los ejercicios.

    Parámetros:
    resultados (dict): Diccionario con las evaluaciones para todos los ejercicios.
    nombre_alumno (str): Nombre del alumno.
    log_file (str): Ruta del archivo de log donde se guardarán los mensajes de error.

    Retorna:
    dict: Un diccionario con los resultados estructurados para todos los ejercicios.
    """
    # Configurar el logging
    configurar_logger(log_file)

    # Inicializar el diccionario para almacenar los resultados
    resultados_detallados = {}
    errores = False

    # Recorrer cada ejercicio y extraer los resultados
    for ejercicio, evaluacion in resultados.items():
        try:
            comentarios = evaluacion  # El texto completo del resultado
            
            # Buscar el patrón para las puntuaciones usando una expresión regular
            puntuaciones_pattern = re.search(r'\*\*Puntuaciones\*\*:\s*(\[[^\]]*\])', comentarios)
            # Buscar el patrón para los comentarios usando una expresión regular
            comentarios_pattern = re.search(r'\*\*Comentarios\*\*:\s*(\[.*?\])', comentarios, re.DOTALL)
            # Buscar el patrón para el comentario general usando una expresión regular
            comentario_general_pattern = re.search(r'\*\*Comentario General\*\*:\s*(\[.*?\])', comentarios, re.DOTALL)
            
            # Si se encuentra el patrón de puntuaciones, evalúa la cadena como una lista
            if puntuaciones_pattern:
                puntuaciones = eval(puntuaciones_pattern.group(1))
            else:
                puntuaciones = []

            # Si se encuentra el patrón de comentarios, evalúa la cadena como una lista
            if comentarios_pattern:
                comentarios_list = eval(comentarios_pattern.group(1))
            else:
                comentarios_list = []

            # Si se encuentra el patrón de comentario general, evalúa la cadena como una lista
            if comentario_general_pattern:
                comentario_general = eval(comentario_general_pattern.group(1))[0]  # Extrae el primer elemento
            else:
                comentario_general = "No disponible"

            # Añadir el resultado al diccionario 'resultados_detallados' para el ejercicio actual
            resultados_detallados[ejercicio] = {
                'puntuaciones': puntuaciones,
                'comentarios': comentarios_list,
                'comentario_general': comentario_general
            }

        except Exception as e:
            error_msg = f"Error procesando los resultados para '{nombre_alumno}' en el ejercicio '{ejercicio}': {e}"
            logging.error(error_msg)
            errores = True

    if not errores:
        success_msg = f"Todos los resultados de '{nombre_alumno}' fueron procesados correctamente."
        logging.info(success_msg)
        print(success_msg)

    return resultados_detallados




## COMPROBAR TODAS LAS FUNCIONES CON UN NOTEBOOK ENTREGADO

In [17]:
import os
import logging
import sys
import io

# Configurar las rutas de los archivos y directorios
directorio_raiz = "/workspace"
directorio_entregas = os.path.join(directorio_raiz, "entregas")
dir_log = os.path.join(directorio_raiz, "logs")
prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
directorio_examen = os.path.join(directorio_raiz, "examenes")
directorio_reports = os.path.join(directorio_raiz, "reports")

log_file = os.path.join(dir_log, 'evaluacion.log')
nombre_fich_examen = 'examen_pruebas.ipynb'
criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
fichero_res = 'resultados_evaluacion.xlsx'
examen_file = os.path.join(directorio_examen, nombre_fich_examen)

# Crear el directorio de logs si no existe
if not os.path.exists(dir_log):
    os.makedirs(dir_log)

# Borrar el archivo de log si ya existe para empezar con un archivo limpio
if os.path.exists(log_file):
    os.remove(log_file)

logger = logging.getLogger(__name__)
# Configurar el logging para registrar errores tanto en consola como en el archivo de log
logging.basicConfig(filename=log_file, level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')
# Registrar un mensaje de log
logging.error("Logging configurado correctamente. Comenzando la ejecución del script.")

try:
    # 1. Cargar criterios
    criterios = cargar_criterios(criterios_file, log_file)
    print("Criterios cargados correctamente. Continuando con la verificación del examen...")

    # 2. Verificar la estructura del examen
    verifica_estructura_examen(examen_file, log_file)
    print("La estructura del examen es correcta. Continuando con la extracción de información...")

    # 3. Extraer la información del examen
    examen_info = extrae_informacion_examen(examen_file, log_file)
    print("Información del examen extraída correctamente. Continuando con el procesamiento de respuestas del alumno...")

    # 4. Procesar respuestas de un alumno
    nombre_fich_alumno = 'santos_alfonso.ipynb'
    alumno_file = os.path.join(directorio_entregas, nombre_fich_alumno)
    respuestas_alumno = procesa_respuestas_alumno(alumno_file, len(examen_info['ejercicios']), log_file)
    print("Respuestas del alumno procesadas correctamente. Continuando con la comprobación de ejecución...")

    # 5. Comprobar la ejecución de las respuestas del alumno
    respuestas_evaluadas = comprueba_ejecucion(respuestas_alumno, log_file)
    print("Ejecución de las respuestas del alumno verificada correctamente.")

    # 6. Evaluar las respuestas utilizando ChatGPT
    evaluaciones = evaluar_ejercicios(examen_info, respuestas_evaluadas, prompt_file, criterios_file, log_file)
    print("Evaluación de las respuestas del alumno completada correctamente.")

    # 7. Extraer los resultados detallados de la evaluación
    resultados_detallados = extraer_resultados(evaluaciones, nombre_fich_alumno, log_file)

    # 8. Imprimir un resumen del resultado de la evaluación detallada
    print("Resumen de la evaluación detallada:")
    for ejercicio, resultado in resultados_detallados.items():
        estado = resultado.get('estado', 'No definido')
        if estado == 'Error':
            mensaje_error = resultado.get('mensaje_de_error', 'Ninguno')
            print(f"{ejercicio}: Estado - Error, Mensaje de error - {mensaje_error}")
        else:
            puntuaciones = resultado.get('puntuaciones', 'N/A')
            comentarios = resultado.get('comentarios', 'N/A')
            comentario_general = resultado.get('comentario_general', 'N/A')
            print(f"{ejercicio}: Puntuaciones - {puntuaciones}, Comentarios - {comentarios}, Comentario General - {comentario_general}")

except RuntimeError as e:
    print(f"Error crítico: {e}")
    sys.exit(1)  # Termina el script con un código de error

except Exception as e:
    print(f"Error inesperado: {e}")
    logging.error(f"Error inesperado: {e}")
    sys.exit(1)  # Termina el script con un código de error


Criterios cargados correctamente. Continuando con la verificación del examen...
La estructura del examen es correcta. Continuando con la extracción de información...
Información del examen extraída correctamente. Continuando con el procesamiento de respuestas del alumno...
Respuestas del alumno procesadas correctamente. Continuando con la comprobación de ejecución...
Ejecución de las respuestas del alumno verificada correctamente.
Evaluación de las respuestas del alumno completada correctamente.
Todos los resultados de 'santos_alfonso.ipynb' fueron procesados correctamente.
Resumen de la evaluación detallada:
Ejercicio 1: Puntuaciones - [10, 10, 5, 8, 9], Comentarios - ['El código se ejecuta sin errores y produce la salida correcta.', 'Se cumplen todas las instrucciones del ejercicio perfectamente.', 'Faltan comentarios que expliquen distintas partes del código, lo que dificultaría la comprensión para un lector.', 'El código es claro y estructurado, aunque el uso de un bucle for es un 

_______________________________________

## Comprobar todas las funciones con todos los ficheros en el directorio

In [18]:
import os
import logging
import sys
import io

# Configurar las rutas de los archivos y directorios
directorio_raiz = "/workspace"
directorio_entregas = os.path.join(directorio_raiz, "entregas")
dir_log = os.path.join(directorio_raiz, "logs")
prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
directorio_examen = os.path.join(directorio_raiz, "examenes")
directorio_reports = os.path.join(directorio_raiz, "reports")

log_file = os.path.join(dir_log, 'evaluacion.log')
nombre_fich_examen = 'examen_pruebas.ipynb'
criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
fichero_res = 'resultados_evaluacion.xlsx'
examen_file = os.path.join(directorio_examen, nombre_fich_examen)

# Crear el directorio de logs si no existe
if not os.path.exists(dir_log):
    os.makedirs(dir_log)

# Borrar el archivo de log si ya existe para empezar con un archivo limpio
if os.path.exists(log_file):
    os.remove(log_file)

# Bloque de preparación: Se ejecuta una sola vez
try:
    # 1. Cargar criterios
    criterios = cargar_criterios(criterios_file, log_file)
    print("Criterios cargados correctamente. Continuando con la verificación del examen...")

    # 2. Verificar la estructura del examen
    verifica_estructura_examen(examen_file, log_file)
    print("La estructura del examen es correcta. Continuando con la extracción de información...")

    # 3. Extraer la información del examen
    examen_info = extrae_informacion_examen(examen_file, log_file)
    print("Información del examen extraída correctamente. Listo para procesar respuestas de los alumnos.")

except RuntimeError as e:
    print(f"Error crítico en la preparación: {e}")
    logging.error(f"Error crítico en la preparación: {e}")
    sys.exit(1)  # Termina el script con un código de error si falla la preparación

# Obtener la lista de alumnos y archivos de notebooks
try:
    alumnos, archivos_notebooks = listar_notebooks(directorio_entregas, log_file)
except RuntimeError as e:
    print(f"Error crítico al listar los notebooks: {e}")
    sys.exit(1)  # Termina el script si no se pueden listar los notebooks

# Bucle para evaluar a cada alumno
for nombre_fich_alumno, archivo_notebook in zip(alumnos, archivos_notebooks):
    try:
        alumno_file = os.path.join(directorio_entregas, archivo_notebook)
        print(f"\nEvaluando al alumno: {nombre_fich_alumno}")

        # 4. Procesar respuestas de un alumno

        respuestas_alumno = procesa_respuestas_alumno(alumno_file, len(examen_info['ejercicios']), log_file)
        print(f"Respuestas del alumno {nombre_fich_alumno} procesadas correctamente. Continuando con la comprobación de ejecución...")

        # 5. Comprobar la ejecución de las respuestas del alumno
        respuestas_evaluadas = comprueba_ejecucion(respuestas_alumno, log_file)
        print(f"Ejecución de las respuestas del alumno {nombre_fich_alumno} verificada correctamente.")

        # 6. Evaluar las respuestas utilizando ChatGPT
        evaluaciones = evaluar_ejercicios(examen_info, respuestas_evaluadas, prompt_file, criterios_file, log_file)
        print(f"Evaluación de las respuestas del alumno {nombre_fich_alumno} completada correctamente.")

        # 7. Extraer los resultados detallados de la evaluación
        resultados_detallados = extraer_resultados(evaluaciones, nombre_fich_alumno, log_file)

        # 8. Imprimir un resumen del resultado de la evaluación detallada
        print(f"Resumen de la evaluación detallada para {nombre_fich_alumno}:")
        for ejercicio, resultado in resultados_detallados.items():
            print(f"{ejercicio}: Puntuaciones - {resultado['puntuaciones']}, Comentarios - {resultado['comentarios']}, Comentario General - {resultado['comentario_general']}")

    except RuntimeError as e:
        print(f"Error al evaluar al alumno {nombre_fich_alumno}: {e}")
        logging.error(f"Error al evaluar al alumno {nombre_fich_alumno}: {e}")
        continue  # Salta al siguiente alumno si hay un error crítico

    except Exception as e:
        print(f"Error inesperado al evaluar al alumno {nombre_fich_alumno}: {e}")
        logging.error(f"Error inesperado al evaluar al alumno {nombre_fich_alumno}: {e}")
        continue  # Salta al siguiente alumno si hay un error inesperado

print("Evaluación completada.")


Criterios cargados correctamente. Continuando con la verificación del examen...
La estructura del examen es correcta. Continuando con la extracción de información...
Información del examen extraída correctamente. Listo para procesar respuestas de los alumnos.

Evaluando al alumno: santos_alfonso
Respuestas del alumno santos_alfonso procesadas correctamente. Continuando con la comprobación de ejecución...
Ejecución de las respuestas del alumno santos_alfonso verificada correctamente.
Evaluación de las respuestas del alumno santos_alfonso completada correctamente.
Todos los resultados de 'santos_alfonso' fueron procesados correctamente.
Resumen de la evaluación detallada para santos_alfonso:
Ejercicio 1: Puntuaciones - [10, 10, 7, 9, 10], Comentarios - ['El código se ejecuta correctamente y produce la salida esperada.', 'Todo el ejercicio se ha cumplido de acuerdo a las instrucciones.', 'Los comentarios son mínimos; sería beneficioso incluir más para mejorar la comprensión del código.', 

In [ ]:
# import logging
# import os
# from joblib import Parallel, delayed
# from tqdm import tqdm

# def main(directorio_raiz, nombre_fich_examen):
#     """
#     Función principal para evaluar los notebooks entregados por los estudiantes.
    
#     Parameters:
#         directorio_raiz (str): Ruta del directorio raíz que contiene los subdirectorios y archivos necesarios.
#         nombre_fich_examen (str): Nombre del archivo del examen.
#     """
#     try:
#         # Configurar las rutas basadas en el directorio raíz
#         directorio_entregas = os.path.join(directorio_raiz, "entregas")
#         dir_log = os.path.join(directorio_raiz, "logs")
#         prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
#         directorio_examen = os.path.join(directorio_raiz, "examenes")
#         directorio_reports = os.path.join(directorio_raiz, "reports")
#         criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
#         fichero_res = 'resultados_evaluacion.xlsx'
#         fichero_examen = os.path.join(directorio_examen, nombre_fich_examen)
#         titulo_examen = "Python Core para Finanzas"

#         # Crear el directorio de logs si no existe
#         if not os.path.exists(dir_log):
#             os.makedirs(dir_log)

#         log_file = os.path.join(dir_log, 'evaluacion.log')
        
#         # Configurar el logging
#         logging.basicConfig(filename=log_file, level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
#         logging.error("Inicio del proceso de evaluación.")

#         # Cargar criterios
#         if not os.path.exists(criterios_file):
#             raise FileNotFoundError(f"El archivo de criterios {criterios_file} no existe.")
#         criterios = cargar_criterios(criterios_file, log_file)

#         # Preprocesar el examen
#         if not os.path.exists(fichero_examen):
#             raise FileNotFoundError(f"El archivo de examen {fichero_examen} no existe.")
#         examen_info = extrae_informacion_examen(fichero_examen, log_file)

#         # Listar los notebooks entregados por los alumnos
#         if not os.path.exists(directorio_entregas):
#             raise FileNotFoundError(f"El directorio de entregas {directorio_entregas} no existe.")
#         alumnos, ficheros = listar_notebooks(directorio_entregas, log_file)
#         if not ficheros:
#             raise FileNotFoundError("No se encontraron notebooks de alumnos en el directorio de entregas.")
        
#         # Inicializar el diccionario de resultados
#         resultados = {}
#         resultados['criterios'] = criterios

#         # Procesar y evaluar cada notebook en paralelo
#         resultados_alumnos = Parallel(n_jobs=-1)(
#             delayed(procesa_y_evalua_notebook)(fich, directorio_entregas, examen_info, criterios, prompt_file, log_file) 
#             for fich in tqdm(ficheros, desc="Procesando notebooks")
#         )

#         # Filtrar resultados exitosos y agregar al diccionario de resultados
#         for result in resultados_alumnos:
#             if result is not None:
#                 alumno, res_extraido = result
#                 resultados[alumno] = res_extraido
        
#         # # Generar los informes en PDF para cada estudiante
#         # generar_pdfs_para_estudiantes(examen_info, directorio_reports, resultados)
        
#         # # Generar el archivo Excel con los resultados de la evaluación
#         # generar_excel_resultados(resultados, filename=os.path.join(directorio_reports, fichero_res))
        
#         print("Proceso completado con éxito.")
#         logging.error("Proceso completado con éxito.")
    
#         return resultados, examen_info

    
#     except Exception as e:
#         error_msg = f"Error en el proceso: {e}"
#         logging.error(error_msg)
#         print(error_msg, file=sys.stderr)

___________________________

## EVALUA TODOS LOS FICHEROS Y GENERA LOS INFORMES DEL PROFESOR Y ALUMNOS


In [19]:
from fpdf import FPDF
import os
import matplotlib.pyplot as plt
import numpy as np

def generar_informe_profesor(examen_info, resultados, nombre_examen, output_dir):
    """
    Genera un informe en PDF para el profesor que resume las calificaciones, comentarios y errores 
    para cada alumno, junto con gráficos de notas.

    Parameters:
        examen_info (dict): Diccionario que contiene el contexto del examen y los enunciados de los ejercicios.
        resultados (dict): Diccionario con los resultados de la evaluación para todos los alumnos.
        nombre_examen (str): Nombre del examen.
        output_dir (str): Directorio donde se guardará el informe en PDF.
    """
    
    # Crear el objeto PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Agregar la portada
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, f"Informe Resumido de Evaluación - {nombre_examen}", ln=True, align='C')
    pdf.ln(20)
    
    # Inicializar las variables para las notas finales
    notas_finales = []
    notas_por_ejercicio = {f'Ejercicio {i+1}': [] for i in range(len(examen_info['ejercicios']))}

    # Resumen para cada alumno
    for alumno, resultado_alumno in resultados.items():
        if alumno == 'criterios':
            continue  # Saltar el diccionario de criterios, ya que no es un alumno
        
        pdf.add_page()
        pdf.set_font("Arial", "B", 14)
        
        # Calcular y añadir la nota final para el alumno al principio
        notas_ejercicios = []
        for i, ejercicio in enumerate(examen_info['ejercicios'], 1):
            resultado_ejercicio = resultado_alumno.get(f"Ejercicio {i}", {})
            puntuaciones = resultado_ejercicio.get('puntuaciones', [])
            if puntuaciones:
                nota_media = sum(puntuaciones) / len(puntuaciones)
                notas_ejercicios.append(nota_media)
        
        if notas_ejercicios:
            nota_final = sum(notas_ejercicios) / len(notas_ejercicios)
            notas_finales.append(nota_final)
            pdf.cell(0, 10, f"Alumno: {alumno.split('.')[0]} - Nota Final: {nota_final:.2f}", ln=True)
        else:
            pdf.cell(0, 10, f"Alumno: {alumno.split('.')[0]}", ln=True)
        
        pdf.ln(10)  # Añadir espacio para evitar superposiciones
        
        # Resumen de cada ejercicio
        for i, ejercicio in enumerate(examen_info['ejercicios'], 1):
            resultado_ejercicio = resultado_alumno.get(f"Ejercicio {i}", {})
            puntuaciones = resultado_ejercicio.get('puntuaciones', [])
            comentarios = resultado_ejercicio.get('comentarios', [])
            
            if puntuaciones and comentarios:
                # Mostrar la puntuación y el comentario para cada criterio
                for j, (puntuacion, comentario) in enumerate(zip(puntuaciones, comentarios)):
                    pdf.set_font("Arial", "", 12)
                    criterio_nombre = examen_info['ejercicios'][i-1]['criterios'][j].strip('@')
                    pdf.cell(0, 10, f"{criterio_nombre}: Puntuación - {puntuacion}", ln=True)
                    pdf.set_font("Arial", "I", 12)
                    pdf.multi_cell(0, 10, f"Comentario - {comentario}")
                
                # Mostrar la nota promedio del ejercicio
                nota_media = sum(puntuaciones) / len(puntuaciones)
                notas_por_ejercicio[f'Ejercicio {i}'].append(nota_media)
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, f"Nota del Ejercicio {i}: {nota_media:.2f}", ln=True)
            else:
                # Si no hay puntuaciones/comentarios, se considera un error
                pdf.set_font("Arial", "", 12)
                pdf.cell(0, 10, "Errores:", ln=True)
                pdf.multi_cell(0, 10, f"{resultado_ejercicio.get('comentarios', 'No disponible')}")
            
            pdf.ln(5)
        
    # Gráfica del histograma de notas finales
    plt.figure(figsize=(6, 4))
    plt.hist(notas_finales, bins=10, edgecolor='black')
    plt.title('Histograma de Notas Finales')
    plt.xlabel('Nota Final')
    plt.ylabel('Número de Alumnos')
    plt.tight_layout()
    hist_path_finales = os.path.join(output_dir, 'histograma_notas_finales.png')
    plt.savefig(hist_path_finales)
    plt.close()

    # Añadir el histograma de notas finales al PDF
    pdf.add_page()
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, 'Histograma de Notas Finales', ln=True)
    pdf.image(hist_path_finales, x=10, y=30, w=190)
    
    # Histograma de las notas y gráficas de criterios por ejercicio
    for i, ejercicio in enumerate(examen_info['ejercicios'], 1):
        notas = notas_por_ejercicio[f'Ejercicio {i}']
        if notas:
            # Histograma de las notas
            plt.figure(figsize=(6, 4))
            plt.hist(notas, bins=10, edgecolor='black')
            plt.title(f'Histograma de Notas - Ejercicio {i}')
            plt.xlabel('Nota')
            plt.ylabel('Número de Alumnos')
            plt.tight_layout()
            img_path = os.path.join(output_dir, f'histograma_ejercicio_{i}.png')
            plt.savefig(img_path)
            plt.close()

            # Añadir el histograma al PDF
            pdf.add_page()
            pdf.set_font("Arial", "B", 14)
            pdf.cell(0, 10, f"Histograma de Notas - Ejercicio {i}", ln=True)
            pdf.image(img_path, x=10, y=30, w=190)

            # Gráfico de barras para las puntuaciones por criterio
            criterios = [criterio.strip('@') for criterio in examen_info['ejercicios'][i-1]['criterios']]

            # Calcular las puntuaciones por criterio manejando el caso de longitud desigual
            puntuaciones_criterio = []
            for j in range(len(criterios)):
                puntuaciones = [
                    resultados[alumno][f'Ejercicio {i}']['puntuaciones'][j]
                    for alumno in resultados
                    if alumno != 'criterios' and f'Ejercicio {i}' in resultados[alumno] and
                    len(resultados[alumno][f'Ejercicio {i}']['puntuaciones']) > j
                ]
                puntuaciones_criterio.append(np.mean(puntuaciones))

            plt.figure(figsize=(6, 4))
            plt.bar(criterios, puntuaciones_criterio, color='blue', edgecolor='black')
            plt.title(f'Puntuaciones por Criterio - Ejercicio {i}')
            plt.xlabel('Criterio')
            plt.ylabel('Puntuación Promedio')
            plt.xticks(rotation=45, ha='right')  # Girar los nombres de los criterios 45 grados
            plt.tight_layout()
            img_path_criterios = os.path.join(output_dir, f'puntuaciones_criterios_ejercicio_{i}.png')
            plt.savefig(img_path_criterios)
            plt.close()

            # Añadir gráfico de puntuaciones por criterio al PDF
            pdf.add_page()
            pdf.image(img_path_criterios, x=10, y=30, w=190)
    
    # Guardar el PDF en el directorio de salida
    output_path = os.path.join(output_dir, f"Informe_Profesor_{nombre_examen}.pdf")
    pdf.output(output_path)
    print(f"Informe resumido generado en: {output_path}")

# Ejemplo de uso:
# nombre_examen = "Python Core para Finanzas"
# generar_informe_profesor(examen_info, resultados, nombre_examen, "/path/to/output/directory")


In [20]:
from fpdf import FPDF
import os
import numpy as np

def generar_informe_pdf_alumnos(examen_info, resultados, nombre_examen, output_dir):
    """
    Genera informes en PDF para todos los alumnos basados en la evaluación de sus ejercicios.

    Parameters:
        examen_info (dict): Diccionario que contiene el contexto del examen y los enunciados de los ejercicios.
        resultados (dict): Diccionario con los resultados de la evaluación para todos los alumnos.
        nombre_examen (str): Nombre del examen.
        output_dir (str): Directorio donde se guardarán los informes en PDF.
    """

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for alumno in resultados.keys():
        if alumno == 'criterios':
            continue

        nombre_alumno = alumno.split('.')[0]

        # Crear el objeto PDF
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)

        # Agregar la portada
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, f"Informe de Evaluación de {nombre_alumno}", ln=True, align='C')
        pdf.ln(10)
        pdf.set_font("Arial", "", 12)
        pdf.cell(0, 10, f"Curso: Introducción a Python para Finanzas", ln=True, align='C')
        pdf.cell(0, 10, f"Examen: {nombre_examen}", ln=True, align='C')
        pdf.cell(0, 10, f"Fecha: {pd.Timestamp('now').strftime('%d/%m/%Y')}", ln=True, align='C')
        pdf.ln(20)

        # Introducción
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 10, examen_info['contexto'])
        pdf.ln(10)

        # Resumen de Evaluación Global
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Resumen de Evaluación Global", ln=True)
        pdf.set_font("Arial", "", 12)

        # Calcular la puntuación total y la nota media
        notas_ejercicios = []
        for i in range(1, len(examen_info['ejercicios']) + 1):
            resultado_ejercicio = resultados[alumno].get(f"Ejercicio {i}", {})
            puntuaciones = resultado_ejercicio.get('puntuaciones', [])
            if puntuaciones:
                nota_media = sum(puntuaciones) / len(puntuaciones)
                notas_ejercicios.append(nota_media)
        
        if notas_ejercicios:
            nota_final = sum(notas_ejercicios) / len(notas_ejercicios)
        else:
            nota_final = 0

        pdf.cell(0, 10, f"Puntuación Total: {nota_final:.2f}", ln=True)
        pdf.ln(5)

        # Comentarios generales según la nota final
        if nota_final >= 9:
            comentarios_generales = "Excelente desempeño. Sigue así para mantener este nivel."
        elif nota_final >= 7:
            comentarios_generales = "Buen desempeño, pero hay algunas áreas que podrían beneficiarse de más práctica y revisión."
        elif nota_final >= 5:
            comentarios_generales = "Desempeño aceptable, pero es necesario trabajar más en ciertos aspectos."
        elif nota_final > 0:
            comentarios_generales = "El desempeño es bajo. Se recomienda revisar los conceptos básicos y practicar más."
        else:
            comentarios_generales = "El alumno no ha completado satisfactoriamente el examen. Es necesario un repaso completo de los temas cubiertos."

        pdf.multi_cell(0, 10, comentarios_generales)
        pdf.ln(10)

        # Detalle de Evaluación por Ejercicio
        for i, ejercicio in enumerate(examen_info['ejercicios'], 1):
            pdf.add_page()  # Cada ejercicio comienza en una nueva página
            pdf.set_font("Arial", "B", 12)

            # Calcular la puntuación del ejercicio
            resultado_ejercicio = resultados[alumno].get(f"Ejercicio {i}", {})
            puntuaciones = resultado_ejercicio.get('puntuaciones', [])
            if puntuaciones:
                nota_media = sum(puntuaciones) / len(puntuaciones)
            else:
                nota_media = 0

            # Título del ejercicio con la puntuación
            pdf.cell(0, 10, f"Ejercicio {i}: {ejercicio['enunciado'].splitlines()[0]}", ln=False)
            pdf.set_x(-50)  # Mueve el cursor para la puntuación hacia la derecha
            pdf.cell(0, 10, f"Puntuación: {nota_media:.2f}", ln=True, align='R')
            pdf.set_font("Arial", "", 12)

            # Enunciado completo
            pdf.set_font("Arial", "B", 12)
            pdf.cell(0, 10, "1. Enunciado:", ln=True)
            pdf.set_font("Arial", "", 12)
            pdf.multi_cell(0, 10, ejercicio['enunciado'])
            pdf.ln(5)

            # Criterios evaluados
            pdf.set_font("Arial", "B", 12)
            pdf.cell(0, 10, "2. Criterios Evaluados:", ln=True)
            pdf.set_font("Arial", "", 12)
            for criterio in ejercicio['criterios']:
                criterio_nombre = criterio.strip('@')
                descripcion_criterio = resultados['criterios'][criterio_nombre]['descripcion']
                pdf.multi_cell(0, 10, f"- {criterio_nombre}: {descripcion_criterio}")
            pdf.ln(5)

            if resultado_ejercicio:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, "3. Comentarios:", ln=True)
                pdf.set_font("Arial", "", 12)
                for comentario in resultado_ejercicio['comentarios']:
                    pdf.multi_cell(0, 10, f"- {comentario}")
                pdf.ln(5)

                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, "4. Comentario General:", ln=True)
                pdf.set_font("Arial", "", 12)
                pdf.multi_cell(0, 10, resultado_ejercicio['comentario_general'])
                pdf.ln(10)
            else:
                pdf.cell(0, 10, "No se encontraron resultados para este ejercicio.", ln=True)
                pdf.ln(10)

        # Conclusión y Recomendaciones ajustadas al nivel del examen
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Conclusión y Recomendaciones", ln=True)
        pdf.set_font("Arial", "", 12)
        
        if nota_final >= 9:
            conclusion = "Has demostrado un excelente dominio del material. Mantén este nivel de esfuerzo y sigue perfeccionando tus habilidades."
        elif nota_final >= 7:
            conclusion = "Buen trabajo, pero hay algunas áreas que podrían beneficiarse de más práctica y revisión."
        elif nota_final >= 5:
            conclusion = "El desempeño es aceptable, pero se recomienda revisar los temas en los que tuviste más dificultades para mejorar en futuras evaluaciones."
        elif nota_final > 0:
            conclusion = "El rendimiento en este examen indica que es necesario un repaso más exhaustivo de los temas cubiertos. Considera buscar apoyo adicional para resolver dudas."
        else:
            conclusion = "Es crucial que dediques tiempo a revisar todos los conceptos clave del curso. Considera buscar ayuda para entender mejor los temas."

        pdf.multi_cell(0, 10, conclusion)
        pdf.ln(10)

        # Guardar el PDF en el directorio de salida
        output_path = os.path.join(output_dir, f"{nombre_alumno}_informe.pdf")
        pdf.output(output_path)
        print(f"Informe generado para {nombre_alumno} en: {output_path}")

# Ejemplo de uso:
# nombre_examen = "Python Core para Finanzas"
# generar_informe_pdf_alumnos(examen_info, resultados, nombre_examen, "/path/to/output/directory")



In [21]:
def guardar_problemas(problemas, output_dir):
    """
    Guarda la lista de problemas en un archivo de texto en el directorio de reports.

    Parameters:
        problemas (list): Lista de problemas encontrados durante la evaluación.
        output_dir (str): Directorio donde se guardará el archivo de problemas.
    """
    problemas_file = os.path.join(output_dir, 'problemas_entregas.txt')
    
    with open(problemas_file, 'w') as f:
        for problema in problemas:
            f.write(f"{problema}\n")
    
    print(f"Archivo de problemas guardado en: {problemas_file}")

In [22]:
def procesa_y_evalua_notebook(fich, directorio_entregas, examen_info, criterios, prompt_file, log_file):
    """
    Procesa y evalúa un notebook individual.
    
    Parameters:
        fich (str): Nombre del archivo del notebook del alumno.
        directorio_entregas (str): Ruta del directorio donde se encuentran los notebooks.
        examen_info (dict): Información del examen.
        criterios (dict): Criterios de evaluación.
        prompt_file (str): Ruta del archivo de prompt.
        log_file (str): Ruta del archivo de log donde se guardarán los mensajes de error.
        
    Returns:
        tuple: Nombre del alumno y resultados de la evaluación.
    """
    try:
        # Procesar respuestas del alumno
        alumno_file = os.path.join(directorio_entregas, fich)
        respuestas_alumno = procesa_respuestas_alumno(alumno_file, len(examen_info['ejercicios']), log_file)
        
        # Comprobar la ejecución de las respuestas
        respuestas_evaluadas = comprueba_ejecucion(respuestas_alumno, log_file)
        
        # Evaluar las respuestas utilizando ChatGPT
        evaluaciones = evaluar_ejercicios(examen_info, respuestas_evaluadas, prompt_file, criterios_file, log_file)
        
        # Extraer los resultados detallados de la evaluación
        resultados_detallados = extraer_resultados(evaluaciones, fich, log_file)
        
        return fich, resultados_detallados
    
    except Exception as e:
        logging.error(f"Error procesando el notebook {fich}: {e}")
        return None

In [26]:
import logging
import os
from joblib import Parallel, delayed
from tqdm import tqdm

def main(directorio_raiz, nombre_fich_examen, nombre_examen):
    try:
        # Configurar las rutas basadas en el directorio raíz
        directorio_entregas = os.path.join(directorio_raiz, "entregas")
        dir_log = os.path.join(directorio_raiz, "logs")
        prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
        directorio_examen = os.path.join(directorio_raiz, "examenes")
        directorio_reports = os.path.join(directorio_raiz, "reports")
        criterios_file = os.path.join(directorio_raiz, 'criterios.txt')
        fichero_res = 'resultados_evaluacion.xlsx'
        fichero_examen = os.path.join(directorio_examen, nombre_fich_examen)

        # Crear el directorio de logs si no existe
        if not os.path.exists(dir_log):
            os.makedirs(dir_log)

        log_file = os.path.join(dir_log, 'evaluacion.log')
        
        # Configurar el logging
        logging.basicConfig(filename=log_file, level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.error("Inicio del proceso de evaluación.")

        # Cargar criterios
        if not os.path.exists(criterios_file):
            raise FileNotFoundError(f"El archivo de criterios {criterios_file} no existe.")
        criterios = cargar_criterios(criterios_file, log_file)

        # Preprocesar el examen
        if not os.path.exists(fichero_examen):
            raise FileNotFoundError(f"El archivo de examen {fichero_examen} no existe.")
        examen_info = extrae_informacion_examen(fichero_examen, log_file)

        # Listar los notebooks entregados por los alumnos
        if not os.path.exists(directorio_entregas):
            raise FileNotFoundError(f"El directorio de entregas {directorio_entregas} no existe.")
        alumnos, ficheros = listar_notebooks(directorio_entregas, log_file)
        if not ficheros:
            raise FileNotFoundError("No se encontraron notebooks de alumnos en el directorio de entregas.")
        
        # Inicializar el diccionario de resultados
        resultados = {}
        resultados['criterios'] = criterios
        problemas = []  # Lista para almacenar los problemas con cada notebook

        # Procesar y evaluar cada notebook en paralelo
        resultados_alumnos = Parallel(n_jobs=-1)(
            delayed(procesa_y_evalua_notebook)(fich, directorio_entregas, examen_info, criterios, prompt_file, log_file) 
            for fich in tqdm(ficheros, desc="Procesando notebooks")
        )

        # Filtrar resultados exitosos y agregar al diccionario de resultados
        procesados_exitosamente = []
        for result in resultados_alumnos:
            if result is not None:
                alumno, res_extraido = result
                resultados[alumno] = res_extraido
                procesados_exitosamente.append(alumno)
            else:
                problemas.append(f"Un notebook no se pudo procesar correctamente.")  # Mejor usar el nombre del archivo si es posible.

        # Generar los informes en PDF para cada estudiante
        generar_informe_pdf_alumnos(examen_info, resultados, nombre_examen, directorio_reports)
        
        # Comparar la lista de notebooks entregados con los informes generados
        informes_generados = os.listdir(directorio_reports)
        for alumno in alumnos:
            informe_esperado = f"{alumno.split('.')[0]}_informe.pdf"
            if informe_esperado not in informes_generados:
                problemas.append(f"{alumno}: No se generó un informe.")

        # Generar el informe para el profesor
        generar_informe_profesor(examen_info, resultados, nombre_examen, directorio_reports)

        # Guardar el archivo con la lista de problemas
        guardar_problemas(problemas, directorio_reports)

        print("Proceso completado con éxito.")
        logging.error("Proceso completado con éxito.")
    
    except Exception as e:
        error_msg = f"Error en el proceso: {e}"
        logging.error(error_msg)
        print(error_msg, file=sys.stderr)
        
        
main("/workspace", "examen.ipynb", "Python Core para Finanzas")

        
        
main("/workspace", "examen.ipynb", "Python Core para Finanzas")

Procesando notebooks: 100%|██████████| 3/3 [00:00<00:00, 4498.72it/s]


Todos los resultados de 'santos_alfonso.ipynb' fueron procesados correctamente.
Todos los resultados de 'ventura_pedro.ipynb' fueron procesados correctamente.
Informe generado para santos_alfonso en: /workspace/reports/santos_alfonso_informe.pdf
Informe generado para ventura_pedro en: /workspace/reports/ventura_pedro_informe.pdf
Informe resumido generado en: /workspace/reports/Informe_Profesor_Python Core para Finanzas.pdf
Archivo de problemas guardado en: /workspace/reports/problemas_entregas.txt
Proceso completado con éxito.


Procesando notebooks: 100%|██████████| 3/3 [00:00<00:00, 4572.28it/s]


Todos los resultados de 'ventura_pedro.ipynb' fueron procesados correctamente.
Todos los resultados de 'santos_alfonso.ipynb' fueron procesados correctamente.
Informe generado para santos_alfonso en: /workspace/reports/santos_alfonso_informe.pdf
Informe generado para ventura_pedro en: /workspace/reports/ventura_pedro_informe.pdf
Informe resumido generado en: /workspace/reports/Informe_Profesor_Python Core para Finanzas.pdf
Archivo de problemas guardado en: /workspace/reports/problemas_entregas.txt
Proceso completado con éxito.


In [93]:
main("/workspace", "examen.ipynb")


Procesando notebooks: 100%|██████████| 3/3 [00:00<00:00, 4390.41it/s]


Todos los resultados de 'ubeda_fernando.ipynb' fueron procesados correctamente.
Todos los resultados de 'ventura_pedro.ipynb' fueron procesados correctamente.
Todos los resultados de 'santos_alfonso.ipynb' fueron procesados correctamente.
Informe resumido generado en: /workspace/reports/Informe_Profesor_Python Core para Finanzas.pdf


Error en el proceso: [Errno 13] Permission denied: 'Python Core para Finanzas'


___________________________

In [89]:
# Ejemplo de uso:
nombre_examen = "Python Core para Finanzas"
generar_informe_pdf_alumnos(examen_info, results, nombre_examen, directorio_reports)

Informe generado para santos_alfonso en: /workspace/reports/santos_alfonso_informe.pdf
Informe generado para ubeda_fernando en: /workspace/reports/ubeda_fernando_informe.pdf
Informe generado para ventura_pedro en: /workspace/reports/ventura_pedro_informe.pdf


In [ ]:
titulo_examen = "Python Core para Finanzas"
generar_informe_profesor(examen_info, results, titulo_examen, directorio_reports)

In [56]:
results['ubeda_fernando.ipynb']

{'Ejercicio 1': {'puntuaciones': [0],
  'comentarios': ['Error en la ejecución: list assignment index out of range'],
  'comentario_general': 'El código presentado contiene errores que impiden su correcta ejecución.'},
 'Ejercicio 2': {'puntuaciones': [0],
  'comentarios': ["Error en la ejecución: cannot access local variable 'maximo' where it is not associated with a value"],
  'comentario_general': 'El código presentado contiene errores que impiden su correcta ejecución.'},
 'Ejercicio 3': {'puntuaciones': [0],
  'comentarios': ["Error en la ejecución: name 'valor_total' is not defined"],
  'comentario_general': 'El código presentado contiene errores que impiden su correcta ejecución.'}}

In [57]:
# Ejemplo de uso:
nombre_alumno = "ubeda_fernando"
nombre_examen = "Primer Examen de Programación"
generar_informe_pdf(nombre_alumno, examen_info, results['ubeda_fernando.ipynb'], nombre_examen, "/workspace/reports")

Informe generado para ubeda_fernando en: /workspace/reports/ubeda_fernando_informe.pdf


_________________________

In [16]:
class PDF(FPDF):
    """
    Clase personalizada para crear reportes de evaluación de ejercicios en formato PDF.
    
    Atributos:
    alumno (str): Nombre del alumno para el cual se está generando el reporte.

    Métodos:
    __init__(self, alumno): Inicializa la instancia de la clase PDF con el nombre del alumno.
    header(self): Añade un encabezado a cada página del PDF con el nombre del alumno.
    footer(self): Añade un pie de página a cada página del PDF con el número de página.
    add_context(self, contexto_examen): Añade el contexto del examen al PDF.
    add_enunciados(self, enunciados): Añade los enunciados de los ejercicios al PDF.
    add_evaluacion(self, alumno, ejercicios, criterios): Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    """
    
    def __init__(self, alumno):
        """
        Inicializa la instancia de la clase PDF con el nombre del alumno.
        
        Parámetros:
        alumno (str): Nombre del alumno para el cual se está generando el reporte.
        """
        super().__init__()
        self.alumno = alumno
    
    def header(self):
        """
        Añade un encabezado a cada página del PDF con el nombre del alumno.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, f'Reporte de Evaluación de Ejercicios de {self.alumno}', 0, 1, 'C')

    def footer(self):
        """
        Añade un pie de página a cada página del PDF con el número de página.
        """
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def add_context(self, contexto_examen):
        """
        Añade el contexto del examen al PDF.
        
        Parámetros:
        contexto_examen (str): Contexto general del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Contexto del Examen:', 0, 1)
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, contexto_examen)
        self.ln(10)

    def add_enunciados(self, enunciados):
        """
        Añade los enunciados de los ejercicios al PDF.
        
        Parámetros:
        enunciados (list): Lista de enunciados de los ejercicios del examen.
        """
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Enunciados de los Ejercicios:', 0, 1)
        self.set_font('Arial', '', 12)
        for i, enunciado in enumerate(enunciados, 1):
            self.cell(0, 10, f'Ejercicio {i}:', 0, 1)
            self.multi_cell(0, 10, enunciado)
            self.ln(5)
        self.ln(10)

    def add_evaluacion(self, alumno, ejercicios, criterios):
        """
        Añade la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
        
        Parámetros:
        alumno (str): Nombre del alumno.
        ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante.
        criterios (list): Lista de criterios de evaluación.
        """
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, f'Informe de {alumno}', 0, 1, 'C')
        self.ln(10)
   
        for ejercicio, contenido in ejercicios.items():
            # Título del ejercicio
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, ejercicio, 0, 1)
            self.ln(5)
            
            # Puntuaciones y comentarios
            for i, criterio in enumerate(criterios):
                self.set_font('Arial', 'B', 12)
                self.cell(0, 10, f'{criterio}:', 0, 1, 'L')
                self.set_font('Arial', '', 12)
                self.multi_cell(0, 10, f'Puntuación: {contenido["puntuaciones"][i]}')
                self.multi_cell(0, 10, f'Comentario: {contenido["comentarios"][i]}')
                self.ln(5)
            
            # Comentario general
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Comentario General del Ejercicio:', 0, 1, 'L')
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, contenido['comentario_general'][0])
            self.ln(10)

In [17]:
# Función para crear el PDF
def create_pdf(file_path, student_name, contexto_examen, enunciados, ejercicios, criterios):
    """
    Crea un archivo PDF con la evaluación del estudiante.

    Parámetros:
    file_path (str): Ruta donde se guardará el archivo PDF.
    student_name (str): Nombre del estudiante.
    contexto_examen (str): Contexto general del examen.
    enunciados (list): Lista de enunciados de los ejercicios del examen.
    ejercicios (dict): Diccionario que contiene las evaluaciones de los ejercicios del estudiante. La estructura del diccionario es:
        {
            'Ejercicio 1': {
                'puntuaciones': [listado de puntuaciones],
                'comentarios': [listado de comentarios],
                'comentario_general': [comentario general]
            },
            'Ejercicio 2': {...},
            ...
        }
    criterios (list): Lista de criterios de evaluación.

    Procedimiento:
    1. Crea una instancia de la clase PDF con el nombre del estudiante.
    2. Añade una página al PDF.
    3. Agrega el contexto del examen al PDF.
    4. Agrega los enunciados de los ejercicios al PDF.
    5. Agrega la evaluación del estudiante al PDF, incluyendo puntuaciones y comentarios para cada criterio.
    6. Guarda el archivo PDF en la ruta especificada.

    Retorna:
    Ninguno
    """
    
    pdf = PDF(student_name)
    pdf.add_page()
    
   
    # Agregar contexto del examen
    pdf.add_context(contexto_examen)
    
    # Agregar enunciados de los ejercicios (comentado porque no está en uso)
    # pdf.add_enunciados(enunciados)
    
    # Agregar evaluación del estudiante
    pdf.add_evaluacion(student_name, ejercicios, criterios)
    
    # Guardar el PDF con el nombre del estudiante
    pdf.output(file_path)

In [18]:
def generar_pdfs_para_estudiantes(examen_preprocesado, directorio_reports, resultados):
    """
    Genera archivos PDF de evaluación para cada estudiante basado en los resultados del examen.

    Parámetros:
    examen_preprocesado (dict): Diccionario con los datos del examen preprocesado que incluye el contexto y los enunciados de los ejercicios.
    directorio_reports (str): Ruta al directorio donde se guardarán los archivos PDF generados.
    resultados (dict): Diccionario que contiene las evaluaciones de los estudiantes. La estructura del diccionario es:
        {
            'nombre_estudiante': {
                'Ejercicio 1': {
                    'puntuaciones': [listado de puntuaciones],
                    'comentarios': [listado de comentarios],
                    'comentario_general': [comentario general]
                },
                'Ejercicio 2': {...},
                ...
            },
            ...
            'criterios': [listado de criterios]
        }

    Precondiciones:
    - El archivo del examen debe existir en la ruta especificada.
    - Las funciones `preprocesa_notebook` y `create_pdf` deben estar definidas previamente.

    Procedimiento:
    1. Crea el directorio para guardar los archivos PDF si no existe.
    2. Para cada estudiante en los resultados, genera un archivo PDF con las evaluaciones y comentarios.
    """

    # Crear directorio para guardar los PDFs
    if not os.path.exists(directorio_reports):
        os.makedirs(directorio_reports)
    
    # Generar el PDF para cada estudiante
    for student_name, ejercicios in resultados.items():
        if student_name != 'criterios':  # Ignorar la clave de criterios
            file_path = os.path.join(directorio_reports, f'{student_name}.pdf')
            try:
                create_pdf(file_path, student_name, examen_preprocesado['contexto_examen'], examen_preprocesado['enunciados_ejercicios'], ejercicios, resultados['criterios'])
            except IndexError as e:
                print(f"Error de índice al procesar {student_name}: {e}")
                raise

In [19]:
def generar_excel_resultados(resultados, filename='resultados_evaluacion.xlsx'):
    filas = []

    for alumno, ejercicios in resultados.items():
        if alumno == 'criterios':  # Ignorar la clave de criterios
            continue
        fila = {'Alumno': alumno}
        notas = []
        for ejercicio, contenido in ejercicios.items():
            media = np.mean(contenido['puntuaciones'])
            fila[ejercicio] = media
            notas.append(media)
        
        # Calcular la nota final como la media ponderada de las notas de los ejercicios
        nota_final = np.mean(notas)
        fila['NOTA FINAL'] = nota_final
        
        filas.append(fila)

    df = pd.DataFrame(filas)
    df.to_excel(filename, index=False)


_______________________________

In [20]:
def evaluar_notebook(fich, directorio_entregas, examen_procesado, prompt_file, criterios, dir_log):
    """
    Procesa y evalúa un notebook de un alumno.
    
    Parameters:
        fich (str): Nombre del archivo del notebook del alumno.
        directorio_entregas (str): Ruta del directorio de entregas.
        examen_procesado (dict): Datos preprocesados del examen.
        prompt_file (str): Ruta del archivo de texto que contiene el prompt.
    
    Returns:
        tuple: Nombre del alumno y resultado de la evaluación extraída.
    """
    try:
        alumno, _ = os.path.splitext(fich)
        nb_preprocesado = preprocesa_respuesta_alumno(os.path.join(directorio_entregas, fich))
        res_eval_tmp = evaluar_ejercicios(examen_procesado, nb_preprocesado, dir_log, prompt_file=prompt_file)
        res_extraido = extraer_resultados(res_eval_tmp, alumno, criterios, dir_log)
        return alumno, res_extraido
    except Exception as e:
        print(f"Error procesando el notebook {fich}: {e}", file=sys.stderr)
        return None

In [21]:
def main(directorio_raiz, nombre_fich_examen):
    """
    Función principal para evaluar los notebooks entregados por los estudiantes.
    
    Parameters:
        directorio_raiz (str): Ruta del directorio raíz que contiene los subdirectorios y archivos necesarios.
        nombre_fich_examen (str): Nombre del archivo del examen.
    """
    try:
        # Construir las rutas basadas en el directorio raíz
        directorio_entregas = os.path.join(directorio_raiz, "entregas")
        dir_log = os.path.join(directorio_raiz, "logs")
        prompt_file = os.path.join(directorio_raiz, 'prompt.txt')
        directorio_examen = os.path.join(directorio_raiz, "examenes")
        directorio_reports = os.path.join(directorio_raiz, "reports")
        fichero_res = 'resultados_evaluacion.xlsx'
        fichero_examen = os.path.join(directorio_examen, nombre_fich_examen)

        # Preprocesar el examen
        if not os.path.exists(fichero_examen):
            raise FileNotFoundError(f"El archivo de examen {fichero_examen} no existe.")
        examen_procesado = preprocesa_examen(fichero_examen) #Extare contexto y enunciados del examen que tienen que estar en celdas markdown
        
        # Listar los notebooks entregados por los alumnos
        if not os.path.exists(directorio_entregas):
            raise FileNotFoundError(f"El directorio de entregas {directorio_entregas} no existe.")
        alumnos, ficheros = listar_notebooks(directorio_entregas)
        if not ficheros:
            raise FileNotFoundError("No se encontraron notebooks de alumnos en el directorio de entregas.")

        # Extraer los criterios de evaluación del prompt
        if not os.path.exists(prompt_file):
            raise FileNotFoundError(f"El archivo de prompt {prompt_file} no existe.")
        criterios = extrae_criterios(prompt_file)

        # Inicializar el diccionario de resultados
        resultados = {}
        resultados['criterios'] = criterios
        
        
        # #Prosecar los notebooks y evaluarlos uno por uno
        # for fich in tqdm(ficheros, desc="Procesando notebooks"):
        #     alumno, resultados_alumnos = evaluar_notebook(fich, directorio_entregas, examen_procesado, prompt_file, criterios, dir_log)
        
        
        
        # Procesar y evaluar cada notebook en paralelo
        resultados_alumnos = Parallel(n_jobs=-1)(
            delayed(evaluar_notebook)(fich, directorio_entregas, examen_procesado, prompt_file, criterios, dir_log) 
            for fich in tqdm(ficheros, desc="Procesando notebooks")
        )

        # Filtrar resultados exitosos y agregar al diccionario de resultados
        for result in resultados_alumnos:
            if result is not None:
                alumno, res_extraido = result
                resultados[alumno] = res_extraido
    
        # Generar los informes en PDF para cada estudiante
        generar_pdfs_para_estudiantes(examen_procesado, directorio_reports, resultados)
        
         # Generar el archivo Excel con los resultados de la evaluación
        generar_excel_resultados(resultados, filename=os.path.join(directorio_reports, fichero_res))
        
        print("Proceso completado con éxito.")

    except Exception as e:
        print(f"Error en el proceso: {e}", file=sys.stderr)
        

# directorio_raiz = "/workspace"
# main(directorio_raiz, "examen.ipynb")
        
       

In [25]:
directorio_raiz = "/workspace"
main(directorio_raiz, "examen.ipynb")

Procesando notebooks: 100%|██████████| 3/3 [00:00<00:00, 4011.13it/s]


Proceso completado con éxito.


______________________________________________________________________

### Pruebas para comparar los nuevos ficheros de prompts para ver si siguen la estrucrura

In [23]:
def validar_prompt(file_path):
    # Leer el contenido del archivo
    with open(file_path, 'r', encoding='utf-8') as file:
        prompt = file.read()

    # Definimos las secciones y su estructura requerida
    seccion_1 = r"\*\*Contexto Examen\*\*\n\{contexto\}\n\n\*\*Descripción del Ejercicio:\*\*\n\{descripcion\}\n\n\*\*Código del estudiante:\*\*\n\{codigo\}"
    seccion_2 = r"Instrucciones Generales para la Evaluación:\n\nEl modelo debe evaluar cada ejercicio utilizando los criterios proporcionados.\nCada criterio debe ser evaluado en una escala de 0 a 10, a menos que el código del ejercicio genere errores al ejecutarse.\nSi el código genera errores, todos los criterios de ese ejercicio deben ser evaluados en una escala de 0 a 5.\nEs importante tener en cuenta que no todos los ejercicios necesitarán usar listas o bucles; en esos casos, los criterios correspondientes deben ser omitidos de la evaluación."
    criterio_patron = r"@@\w+@@\nDescripción: .+\nEjemplo: .+"
    seccion_3 = (
        r"Devuelve tres listas solo con la lista proporcionada en formato y nada más:\n\n"
        r"A. \*\*Puntuaciones\*\*: Una lista de puntuaciones \(solo los números, de 0 a 10\) correspondiente a cada criterio en el orden en que se presentan. Si el código da algún error al ejecutarse, la nota máxima para cada criterio será 5.\n   - Formato: \[0, 10, 7, \.\.\.\]\n"
        r"B. \*\*Comentarios\*\*: Una lista de comentarios correspondiente a cada criterio en el mismo orden.\n   - Formato: \[\"Comentario para exactitud\", \"Comentario para claridad\", \.\.\.\]\n"
        r"C. \*\*Comentario General\*\*: Un comentario que ofrezca una idea global sobre el ejercicio teniendo en cuenta los criterios definidos. Especifica claramente si el código genera algún error al ejecutarse.\n   - Formato: \[\"Comentario general sobre el ejercicio\"\]"
    )

    # Validar secciones
    if not re.search(seccion_1, prompt):
        return "La primera sección no sigue la estructura requerida."

    if not re.search(seccion_2, prompt):
        return "La segunda sección no sigue la estructura requerida."

    # Validar al menos un criterio en la segunda sección
    criterios = re.findall(criterio_patron, prompt)
    if not criterios:
        return "No se encontraron criterios en la segunda sección."

    # Validar la tercera sección
    if not re.search(seccion_3, prompt):
        return "La tercera sección no sigue la estructura requerida."

    return "El prompt es válido."

In [24]:
#validar_prompt("/workspace/prompt funciona copy.txt")